In [1]:
import os
import numpy as np
import re
import shutil
import tensorflow as tf
DATA_DIR = "./data"
CHECKPOINT_DIR = os.path.join(DATA_DIR, "checkpoints")

In [12]:
%ls

 D 드라이브의 볼륨: Home
 볼륨 일련 번호: C446-298C

 D:\Home\Project\MachineLearning\DeepLearning\Chapter05 디렉터리

2024-08-16  오후 05:25    <DIR>          .
2024-08-16  오후 05:22    <DIR>          ..
2024-08-16  오후 05:23    <DIR>          .ipynb_checkpoints
2024-08-16  오후 05:24    <DIR>          data
2024-08-16  오후 05:25             1,920 OneToMany01.ipynb
               1개 파일               1,920 바이트
               4개 디렉터리  907,754,565,632 바이트 남음


In [16]:
def download_and_read(urls):
    texts = []
    for i, url in enumerate(urls):
        p = tf.keras.utils.get_file("ex1-{:d}.txt".format(i), url,
            cache_dir=".")
        text = open(p, mode="r", encoding="utf-8").read()
        # remove byte order mark
        text = text.replace("\ufeff", "")
        # remove newlines
        text = text.replace('\n', ' ')
        text = re.sub(r'\s+', " ", text)
        # add it to the list
        texts.extend(text)
    return texts

texts = download_and_read([
    "http://www.gutenberg.org/cache/epub/28885/pg28885.txt",
    "https://www.gutenberg.org/files/12/12-0.txt"
])

177660/177660 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step
196464/196464 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step
